In [4]:
import numpy as np
import pandas as pd

# Read the data
data = pd.read_csv("ncr_ride_bookings.csv")
data_copy = data.copy()


# ======================================================
# 📌 Data Cleaning 
# ======================================================

data.fillna({"Avg VTAT": data["Avg VTAT"].median(),
             "Avg CTAT": data["Avg CTAT"].median(),
             "Cancelled Rides by Customer": 0,
             "Reason for cancelling by Customer": "Not Applicable",
             "Cancelled Rides by Driver": 0,
             "Driver Cancellation Reason": "Not Applicable",
             "Incomplete Rides": 0,
             "Incomplete Rides Reason": "Not Applicable",
             "Booking Value": 0,
             "Ride Distance": 0,
             "Payment Method": "Not Applicable"             
}, inplace=True)


# ======================================================
# 📌 1. Booking Overview
# ======================================================

booking_counts = data["Booking Status"].value_counts()
booking_percentage = data["Booking Status"].value_counts(normalize=True)*100
booking_overview = pd.DataFrame({"Booking": booking_counts, "Percentage": booking_percentage})
total_bookings = len(data)
print("*" *140)
print("📌  Booking Overview".center(120))
print("*" * 140)
print(booking_overview.to_string())
print("-"*50)
print(f"{"Total Bookings":<24} {total_bookings}")

# Bookings by date
data["Date"] = pd.to_datetime(data["Date"], format="%m/%d/%Y", errors="coerce")
bookings_by_date = data.groupby(data["Date"].dt.date).size()
#display(bookings_by_date.head(10))

# Bookings by time
data["Time"] = pd.to_datetime(data["Time"], format="%H:%M:%S", errors="coerce")
data["Hour"] = data["Time"].dt.hour                    # 0–23
data["AM/PM"] = data["Time"].dt.strftime("%I %p")  # Pretty labels

# Sorting Time in correct order
#bookings_by_hour = (data.groupby(["Hour", "AMPM"]).size().reset_index(name="Bookings").sort_values("Hour"))
#display(bookings_by_hour[["Hour", "AMPM", "Bookings"]].style.hide(axis="index"))
# Bookings by time on date
#bookings_by_date_hour = data.groupby([data["Date"].dt.date, "Hour", "AMPM"]).size().reset_index(name="Bookings").sort_values(["Date", "Hour"])

# Peak booking hours (top 5)
print()
print("-"*40)
print("Peak Hours".center(40))
print("-"*40)

peak_hours = (
    data.groupby(["Hour", "AM/PM"])
        .size()
        .reset_index(name="Bookings")
        .sort_values("Bookings", ascending=False)
        .head(5)
)
display(peak_hours[["AM/PM", "Bookings"]].head(3).style.hide(axis="index"))

# Peak Day of week (0=Monday, 6=Sunday)
print()
print("-"*40)
print("Peak Day of week".center(40))
print("-"*40)

data["DayOfWeek"] = data["Date"].dt.day_name()

peak_days = (
    data.groupby("DayOfWeek")
        .size()
        .reset_index(name="Bookings")
        .sort_values("Bookings", ascending=False)
)
display(peak_days.head(3).style.hide(axis="index"))

# Peak Month 
print()
print("-"*40)
print("Peak Month".center(40))
print("-"*40)

data["Month"] = data["Date"].dt.month_name()

peak_months = (
    data.groupby("Month")
        .size()
        .reset_index(name="Bookings")
        .sort_values("Bookings", ascending=False)
)
display(peak_months.head(3).style.hide(axis="index"))

# Time-of-day buckets
# Define custom time-of-day ranges
print()
print("-"*40)
print("Time-of-day buckets".center(40))
print("-"*40)

def time_of_day(hour):
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

data["TimeOfDay"] = data["Hour"].apply(time_of_day)
peak_times = data.groupby("TimeOfDay").size().reset_index(name="Bookings").sort_values("Bookings", ascending=False)
display(peak_times.style.hide(axis="index"))

# ======================================================
# 📌 2. Customer Behavior
# ======================================================
print("*" *140)
print("📌 Customer Behavior".center(120))
print("*" * 140)
print()
# Count number of bookings per customer
booking_counts_by_customer = data.groupby(data["Customer ID"]).size().reset_index(name="Bookings")
repeated_customers = booking_counts_by_customer[booking_counts_by_customer["Bookings"]>1]
new_customers = booking_counts_by_customer[booking_counts_by_customer["Bookings"]==1]

print(f"{"Repeated Customers :":<10} {len(repeated_customers)}")
print(f"{"New Customers :":<20} {len(new_customers)}")


# Reason for cancellation
print("-"*80)
print("Customer Cancellation Summary".center(80))
print("-"*80)
cancel_by_customer = data["Reason for cancelling by Customer"].value_counts()
customer_cancellation = cancel_by_customer.drop("Not Applicable")
cancellation_summary = customer_cancellation.reset_index()
cancellation_summary.columns = ["Reason for Cancellation", "Count"]
cancellation_summary["Percentage"] = ((cancellation_summary["Count"]/cancellation_summary["Count"].sum()) * 100).round(2)
cancellation_summary = cancellation_summary.sort_values(by="Percentage", ascending=False)
print(cancellation_summary.to_string(index=False))
print()
print("Total cancellation by Customer :", cancellation_summary["Count"].sum())
print()


# Payment method preferences
print("-"*80)
print("Payment Method Summary".center(80))
print("-"*80)
payment_method = data["Payment Method"].value_counts()
payment_method = payment_method.drop("Not Applicable")
payment_summary = payment_method.reset_index()
payment_summary.columns = ["Payment Method", "Count"]
payment_summary["Percentage"] = ((payment_summary["Count"]/payment_summary["Count"].sum())*100).round(2)
payment_summary = payment_summary.sort_values(by="Percentage", ascending=False)
print(payment_summary.to_string(index=False))
print()


# ======================================================
# 📌 3. Driver Behavior
# ======================================================
print("*" *140)
print("📌 Driver Behavior".center(120))
print("*" * 140)
print()
# Reason for cancellation
print("-"*80)
print("Driver Cancellation Summary".center(80))
print("-"*80)

driver_cancellation = data["Driver Cancellation Reason"].value_counts()
driver_cancellation = driver_cancellation.drop("Not Applicable")
driver_cancellation_summary = driver_cancellation.reset_index()
driver_cancellation_summary.columns = ["Cancellation Reason", "Count"]
driver_cancellation_summary["Percentage"] = ((driver_cancellation_summary["Count"]/driver_cancellation_summary["Count"].sum())*100).round(2)
driver_cancellation_summary = driver_cancellation_summary.sort_values(by="Percentage", ascending=False)
print(driver_cancellation_summary.to_string(index=False))
print()
print("Total cancellation by Driver :", driver_cancellation_summary["Count"].sum())
print()

# Incomplete Rides

print("-"*60)
print("Incomplete Rides Summary".center(60))
print("-"*60)
incomplete_rides = data["Incomplete Rides Reason"].value_counts()
incomplete_rides = incomplete_rides.drop("Not Applicable")
incomplete_rides_summary = incomplete_rides.reset_index()
incomplete_rides_summary.columns = ["Incomplete Rides Reason", "Count"]
incomplete_rides_summary["Percentage"] = ((incomplete_rides_summary["Count"]/incomplete_rides_summary["Count"].sum())*100).round(2)
incomplete_rides_summary = incomplete_rides_summary.sort_values(by="Percentage", ascending=False)
print(incomplete_rides_summary.to_string(index=False))
print()
print("Total Incomplete Rides :", incomplete_rides_summary["Count"].sum())
print()


# ======================================================
# 📌 4. Ride Performance
# ======================================================
print("="*140)
print("📌 Ride Performance".center(120))
print("="*140)

# Average VTAT & CTAT
print()
print(f"{'Average Vehicle Turnaround Time  :':<25} {data["Avg VTAT"].mean().round(2)}")
print(f"{'Average Customer Turnaround Time :':<25} {data["Avg CTAT"].mean().round(2)}")
print(f"{'Average Ride Distance :':<25} {data["Ride Distance"].mean().round(2)}")
print(f"{'Average Fare :':<25} {data["Booking Value"].mean().round(2)}")
print()

# ======================================================
# 📌 5. Service Quality
# ======================================================

print("="*140)
print("📌 Service Quality".center(120))
print("="*140)
print()
print(f"{'Average Driver Rating   :'} {data['Driver Ratings'].mean().round(2)}")
print(f"{'Average Customer Rating :'} {data['Customer Rating'].mean().round(2)}")
print()
print("-"*60)
print("Average Driver Ratings by Vehicle Type".center(60))
print("-"*60)
print(f"{data.groupby("Vehicle Type")["Driver Ratings"].mean().sort_values(ascending=False).round(2)}")
print()
print("-"*60)
print("Average Customer Ratings by Vehicle Type".center(60))
print("-"*60)
print(f"{data.groupby("Vehicle Type")["Customer Rating"].mean().sort_values(ascending=False).round(2)}")


# ======================================================
# 📌 6. Vehicle Type Analysis
# ======================================================

print("="*140)
print("📌 Vehicle Type Analysis".center(120))
print("="*140)
print()
print("-"*60)
print("Completed Rides".center(60))
print("-"*60)
completed_rides_by_vehicle = data.groupby("Vehicle Type")["Booking Status"].apply(lambda x: (x =="Completed").sum())
vehicle_summary = completed_rides_by_vehicle.reset_index()
vehicle_summary.columns = ["Vehicle Type", "Completed Rides"]
vehicle_summary["Percentage"] = ((vehicle_summary["Completed Rides"]/vehicle_summary["Completed Rides"].sum())*100).round(2)
vehicle_summary = vehicle_summary.sort_values(by="Percentage", ascending=False)
print(vehicle_summary.to_string(index=False))
print()
print(f"{"Total Completed Rides  :"} {vehicle_summary["Completed Rides"].sum()}")
print()
print("-"*60)
print("Canceled Rides".center(60))
print("-"*60)
canceled_rides_by_vehicle = data.groupby("Vehicle Type")["Booking Status"].apply(lambda x: (x !="Completed").sum())
vehicle_summary = canceled_rides_by_vehicle.reset_index()
vehicle_summary.columns = ["Vehicle Type", "Canceled Rides"]
vehicle_summary["Percentage"] = ((vehicle_summary["Canceled Rides"]/vehicle_summary["Canceled Rides"].sum())*100).round(2)
vehicle_summary = vehicle_summary.sort_values(by="Percentage", ascending=False)
print(vehicle_summary.to_string(index=False))
print()
print(f"{"Total Canceled Rides  :"} {vehicle_summary["Canceled Rides"].sum()}")
print()


# ======================================================
# 📌 7. Location Analysis (Top Pickup & Drop Points)
# ======================================================

print("="*140)
print("📌 Location Analysis (Top Pickup & Drop Points)".center(120))
print("="*140)
print()
print("-"*60)
print("Top 10 Pickup Locations".center(60))
print("-"*60)
print(f"{data["Pickup Location"].value_counts().sort_values(ascending=False).head(10)}")
print()
print("-"*60)
print("Top 10 Drop Locations".center(60))
print("-"*60)
print(f"{data["Drop Location"].value_counts().sort_values(ascending=False).head(10)}")
print()
print("-"*60)
print("Top 10 Pickup-Drop Combinations".center(60))
print("-"*60)
pickup_drop = data.groupby(["Pickup Location", "Drop Location"]).size().reset_index(name="Count")
print(pickup_drop.sort_values("Count", ascending=False).head(10).to_string(index=False))
print()


# ======================================================
# 📌 8. Monthly Revenue Trend by Vehicle Type
# ======================================================
import calendar

# Complete & Incomplete rides summary
completed = data[data["Booking Status"]== "Completed"].groupby("Month").agg(Completed_Rides = ("Booking Status", "count"), Rides_Distance = ("Ride Distance", "sum"), Completed_Rides_Revenue = ("Booking Value", "sum"))
incomplete = data[data["Booking Status"]== "Incomplete"].groupby("Month").agg(Incomplete_Rides = ("Booking Status", "count"), Ride_Distance = ("Ride Distance", "sum"), Incomplete_Rides_Revenue = ("Booking Value", "sum"))

# Monthly Summary
month_order = list(calendar.month_name[1:])
month_order.append("Total") 
monthly_summary = completed.merge(incomplete, on="Month", how="outer").fillna(0)
monthly_summary = monthly_summary.reset_index()
monthly_summary["Total Revenue"] = (monthly_summary["Completed_Rides_Revenue"] + monthly_summary["Incomplete_Rides_Revenue"])


monthly_summary["Month"] = pd.Categorical(monthly_summary["Month"], categories=month_order, ordered=True)
monthly_summary = monthly_summary.sort_values("Month")

total = pd.DataFrame({ "Month": ["Total"],
                       "Completed_Rides": [monthly_summary["Completed_Rides_Revenue"].sum()],
                       "Rides_Distance": [monthly_summary["Rides_Distance"].sum()],
                       "Completed_Rides_Revenue": [monthly_summary["Completed_Rides_Revenue"].sum()],
                       "Incomplete_Rides": [monthly_summary["Incomplete_Rides"].sum()],
                       "Ride_Distance": [monthly_summary["Ride_Distance"].sum()],
                       "Incomplete_Rides_Revenue": [monthly_summary["Incomplete_Rides_Revenue"].sum()],
                       "Total Revenue": [monthly_summary["Total Revenue"].sum()]    
})
monthly_summary = pd.concat([monthly_summary, total])

# Format to 2 decimals with rupee symbol
format_rupee = lambda x: f"₹{x:,.2f}"

for col in ["Completed_Rides_Revenue", "Incomplete_Rides_Revenue", "Total Revenue"]:
    monthly_summary[col] = monthly_summary[col].apply(format_rupee)

format_distance= lambda y: f"{y:.2f}"

for cols in ["Completed_Rides", "Rides_Distance", "Incomplete_Rides", "Ride_Distance"]:
    monthly_summary[cols] = monthly_summary[cols].apply(format_distance)


print()
print("="*140)
print("📌 Monthly Revenue Summary".center(140))
print("="*140)
print()
display(monthly_summary.style.hide(axis="index"))



print()
print("="*140)
print("📌 Monthly Revenue by Vehicle Type".center(140))
print("="*140)
print()


revenue_by_vehicle = pd.pivot_table( data,
                                     index = "Month",
                                     columns = "Vehicle Type",
                                     values = "Booking Value",
                                     aggfunc = "sum",
                                     fill_value = 0).reset_index()

revenue_by_vehicle["Month"] = pd.Categorical(revenue_by_vehicle["Month"], categories = month_order, ordered=True)
revenue_by_vehicle["Total Revenue"] = revenue_by_vehicle.drop(columns=["Month"]).sum(axis=1)
revenue_by_vehicle = revenue_by_vehicle.sort_values("Month")

total_row = revenue_by_vehicle.drop(columns=["Month"]).sum()
total_row["Month"] = "Total"

revenue_by_vehicle = pd.concat([revenue_by_vehicle, pd.DataFrame([total_row])], ignore_index=True)

for col in revenue_by_vehicle.columns:
    if col !="Month":
        revenue_by_vehicle[col]= revenue_by_vehicle[col].apply(lambda x: f"₹{x:,.2f}")
display(revenue_by_vehicle.style.hide(axis="index"))


with pd.ExcelWriter("ncr_analysis_output.xlsx", engine="xlsxwriter") as writer:
    revenue_by_vehicle.to_excel(writer, sheet_name= "Monthly Revenue by Vehicle", startrow=0 )
    monthly_summary.to_excel(writer, sheet_name= "Monthly Summary", startrow=0)



    

********************************************************************************************************************************************
                                                  📌  Booking Overview                                                   
********************************************************************************************************************************************
                       Booking  Percentage
Booking Status                            
Completed                93000        62.0
Cancelled by Driver      27000        18.0
No Driver Found          10500         7.0
Cancelled by Customer    10500         7.0
Incomplete                9000         6.0
--------------------------------------------------
Total Bookings           150000

----------------------------------------
               Peak Hours               
----------------------------------------


AM/PM,Bookings
06 PM,12397
07 PM,11047
05 PM,11044



----------------------------------------
            Peak Day of week            
----------------------------------------


DayOfWeek,Bookings
Monday,21644
Saturday,21542
Wednesday,21413



----------------------------------------
               Peak Month               
----------------------------------------


...

TimeOfDay,Bookings
Morning,45458
Evening,44118
Afternoon,37342
Night,23082


********************************************************************************************************************************************
                                                  📌 Customer Behavior                                                   
********************************************************************************************************************************************

Repeated Customers : 1206
New Customers :      147582
--------------------------------------------------------------------------------
                         Customer Cancellation Summary                          
--------------------------------------------------------------------------------
                     Reason for Cancellation  Count  Percentage
                               Wrong Address   2362       22.50
                             Change of plans   2353       22.41
Driver is not moving towards pickup location   2335       22.24
                      Driver asked to cance

Month,Completed_Rides,Rides_Distance,Completed_Rides_Revenue,Incomplete_Rides,Ride_Distance,Incomplete_Rides_Revenue,Total Revenue
January,7951.00,207020.15,"₹4,001,547.00",808.00,8284.86,"₹409,522.00","₹4,411,069.00"
February,7368.00,189703.73,"₹3,756,610.00",681.00,7091.40,"₹329,180.00","₹4,085,790.00"
March,7954.00,207750.51,"₹4,174,900.00",727.00,7533.08,"₹393,288.00","₹4,568,188.00"
April,7632.00,197405.46,"₹3,885,255.00",725.00,7585.34,"₹368,534.00","₹4,253,789.00"
May,7905.00,203340.84,"₹3,935,207.00",768.00,8200.48,"₹385,472.00","₹4,320,679.00"
June,7757.00,202713.00,"₹3,964,298.00",738.00,7845.17,"₹361,362.00","₹4,325,660.00"
July,7926.00,206404.65,"₹3,961,939.00",803.00,8590.23,"₹403,984.00","₹4,365,923.00"
August,7780.00,200963.09,"₹3,867,753.00",735.00,7705.26,"₹375,756.00","₹4,243,509.00"
September,7542.00,196233.61,"₹3,820,536.00",743.00,7831.28,"₹370,857.00","₹4,191,393.00"
October,7905.00,207109.78,"₹4,008,403.00",761.00,8254.56,"₹408,767.00","₹4,417,170.00"



                                                     📌 Monthly Revenue by Vehicle Type                                                      



Month,Auto,Bike,Go Mini,Go Sedan,Premier Sedan,Uber XL,eBike,Total Revenue
January,"₹1,059,174.00","₹684,389.00","₹891,180.00","₹789,039.00","₹558,325.00","₹122,468.00","₹306,494.00","₹4,411,069.00"
February,"₹1,008,107.00","₹621,189.00","₹837,301.00","₹747,544.00","₹486,173.00","₹108,570.00","₹276,906.00","₹4,085,790.00"
March,"₹1,157,233.00","₹688,752.00","₹882,507.00","₹818,031.00","₹548,171.00","₹144,538.00","₹328,956.00","₹4,568,188.00"
April,"₹1,089,526.00","₹617,289.00","₹859,005.00","₹764,755.00","₹485,985.00","₹141,328.00","₹295,901.00","₹4,253,789.00"
May,"₹1,114,126.00","₹622,624.00","₹830,104.00","₹797,838.00","₹522,202.00","₹131,025.00","₹302,760.00","₹4,320,679.00"
June,"₹1,047,547.00","₹652,375.00","₹839,418.00","₹829,298.00","₹522,063.00","₹129,831.00","₹305,128.00","₹4,325,660.00"
July,"₹1,053,010.00","₹692,294.00","₹831,950.00","₹807,746.00","₹578,005.00","₹130,947.00","₹271,971.00","₹4,365,923.00"
August,"₹1,065,155.00","₹662,192.00","₹823,639.00","₹748,277.00","₹490,333.00","₹131,829.00","₹322,084.00","₹4,243,509.00"
September,"₹1,070,561.00","₹658,929.00","₹852,310.00","₹703,379.00","₹508,303.00","₹111,974.00","₹285,937.00","₹4,191,393.00"
October,"₹1,082,162.00","₹660,783.00","₹922,650.00","₹788,626.00","₹540,471.00","₹126,828.00","₹295,650.00","₹4,417,170.00"
